In [1]:
%%javascript
$('#appmode-leave').hide();
$('#visit-repo-link').hide();

<IPython.core.display.Javascript object>

# Rate law notebook

Copyright **Peter Kraus and Paolo Raiteri**, January 2021

## Determination of the Rate Law \#1
Imagine to perform a series of experiments where you mix nitrogen oxide, nitrogen dioxide and molecular oxygen to obtain gaseous di-nitrogen pentoxyde, according the the following reaction
\begin{equation*}
NO (g)   +   NO_2  (g)   +  O_2 (g) \to   N_2O_5 (g)
\end{equation*}

Determine the rate law and the rate constant for the reaction.\
Tip: use the isolation method first to determine the rate law

### Instructions:

- Adjust the concentration of $NO$, $NO_2$ and  $O_2$ using the sliders below, 
- Click `Calculate` to run the virtual experiment and obtain the reaction rate,
- Click `Download CSV` to export the complete data set for all the experiments as a CSV file.

In [2]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
from IPython.display import display, Markdown

In [3]:
# set kinetic parameters
with open("rate_parameters.json") as infile:
    jsdata = json.load(infile)

params = jsdata["test"]


In [4]:
# delete existing result file and setup rng
if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
    os.remove(os.path.join(os.getcwd(), "results.csv"))

#random.seed(params["error"].get("seed", 0))
t = int( time.time() * 1000.0 )
random.seed( ((t & 0xff000000) >> 24) +
             ((t & 0x00ff0000) >>  8) +
             ((t & 0x0000ff00) <<  8) +
             ((t & 0x000000ff) << 24)   )


In [10]:
out_R = ipw.Label(value="Click Calculate.")
out_P = ipw.Output()
out_L = ipw.Output()

species_A = "[NO] (mol/l)"
species_B = "[NO$_2$] (mol/l)"
species_C = "[O$_2$] (mol/l)"

result = "R (mol/l·s)"

with out_L:
    display(Markdown("[Download CSV](./results.csv)"))

def calc(btn):
    R = params["k"] * conc_A.value ** params["x"] * conc_B.value ** params["y"]
    # Random error
    dR = R * params["error"].get("random", 0) * (0.5 - random.random()) * 2 
    # Systematic error
    dR = dR + R * params["error"].get("systematic", 0)
    out_R.value = f"{R + dR:.6e}"
    out_P.clear_output()
    try:
        res = pd.read_csv("results.csv") 
    except FileNotFoundError:
        res = pd.DataFrame(columns=[species_A, 
                                    species_B, 
                                    species_C, 
                                    result])
    res = res.append({species_A: conc_A.value, 
                      species_B: conc_B.value, 
                      species_C: conc_C.value, 
                      result: R + dR}, 
                     ignore_index=True)
    res.to_csv("results.csv", index=False)
    with out_P:
        display(res.tail(50))

def reset(btn):
    if os.path.exists(os.path.join(os.getcwd(), "results.csv")):
        os.remove(os.path.join(os.getcwd(), "results.csv"))
    res = pd.DataFrame(columns=[species_A, 
                                species_B, 
                                species_C, 
                                result])
    res.to_csv("results.csv", index=False)
    with out_P:
        out_P.clear_output()
        display(res.tail(50))


conc_A = ipw.FloatLogSlider(value=params["[A]"], min=-3, max=0)
conc_B = ipw.FloatLogSlider(value=params["[B]"], min=-3, max=0)
conc_C = ipw.FloatLogSlider(value=params["[C]"], min=-3, max=0)

btn_reset = ipw.Button(description="Restart Laboratory", layout=ipw.Layout(width="150px"))
btn_reset.on_click(reset)


btn_calc = ipw.Button(description="Calculate Rate", layout=ipw.Layout(width="150px"))
btn_calc.on_click(calc)

rows = []
rows.append(ipw.HBox([ipw.Label(value="Concentration of NO:", layout=ipw.Layout(width="150px")), 
                      conc_A, 
                      ipw.Label(value="mol/l")]))
rows.append(ipw.HBox([ipw.Label(value="Concentration of NO$_2$:", layout=ipw.Layout(width="150px")), 
                      conc_B, 
                      ipw.Label(value="mol/l")]))
rows.append(ipw.HBox([ipw.Label(value="Concentration of O$_2$:", layout=ipw.Layout(width="150px")), 
                      conc_C, 
                      ipw.Label(value="mol/l")]))
rows.append(ipw.HBox([ipw.Label(value="Initial reaction rate is:", layout=ipw.Layout(width="150px")), 
                      out_R,
                      ipw.Label(value="mol/l·s")]))
rows.append(ipw.HBox([btn_reset, btn_calc, out_L]))
rows.append(ipw.HBox([out_P]))
#calc(btn_calc)
ipw.VBox(rows)